In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = "https://www.rmc-cmr.ca/en/registrars-office/mechanical-aeronautical-engineering-undergraduate-programmes"

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

In [4]:
driver.get(url)

# 1. Collect course link texts for webdriver to click on

In [5]:
page_soup = soup(driver.page_source, 'lxml')

In [6]:
tables = page_soup.findAll("table", {"class": "table table-bordered table-condensed"})[:2]

In [7]:
link_texts = []
for table in tables:
    containers = table.findAll("a", {"class": None})
    for container in containers:
        link_texts.append(container.text)
        
len(link_texts)

62

In [8]:
link_texts = list(dict.fromkeys(link_texts))
link_texts

['ENE100',
 'PSE103',
 'CSE101',
 'CCE101',
 'MAE101',
 'PHE104',
 'ATE101',
 'LCF',
 'MAE129',
 'GEE167',
 'HIE207',
 'MAE226',
 'PHE205',
 'MEE233',
 'GEE241',
 'GEE293',
 'HIE203',
 'POE205',
 'MAE209',
 'MAE227',
 'MEE231',
 'MEE245',
 'PSE301',
 'MAE328',
 'MEE311',
 'MEE331',
 'MEE333',
 'MEE351',
 'ATE301',
 'HIE271',
 'MEE301',
 'MEE303',
 'MEE313',
 'MEE346',
 'MEE353',
 'GEE393',
 'HIE289',
 'MEE471',
 'MEE407',
 'MEE421',
 'MEE431',
 'MEE443',
 'MEE482',
 'PSE401']

In [9]:
#scrape technical electives
containers = page_soup.find("aside", {"id": "wb-auto-1"}).find("ul", {"class": "list-unstyled"}).findAll("a")
link_texts1 = [container.text for container in containers]
len(link_texts1)

16

In [10]:
link_texts = link_texts + link_texts1
link_texts

['ENE100',
 'PSE103',
 'CSE101',
 'CCE101',
 'MAE101',
 'PHE104',
 'ATE101',
 'LCF',
 'MAE129',
 'GEE167',
 'HIE207',
 'MAE226',
 'PHE205',
 'MEE233',
 'GEE241',
 'GEE293',
 'HIE203',
 'POE205',
 'MAE209',
 'MAE227',
 'MEE231',
 'MEE245',
 'PSE301',
 'MAE328',
 'MEE311',
 'MEE331',
 'MEE333',
 'MEE351',
 'ATE301',
 'HIE271',
 'MEE301',
 'MEE303',
 'MEE313',
 'MEE346',
 'MEE353',
 'GEE393',
 'HIE289',
 'MEE471',
 'MEE407',
 'MEE421',
 'MEE431',
 'MEE443',
 'MEE482',
 'PSE401',
 'MEE401',
 'MEE404',
 'MEE417',
 'MEE423',
 'MEE425',
 'MEE435',
 'MEE437',
 'MEE438',
 'MEE446',
 'MEE451',
 'MEE469',
 'AEE435',
 'AEE461',
 'AEE463',
 'AEE467',
 'AEE491']

# 2. Test run - try to scrape the first course

In [11]:
driver.find_element_by_link_text(link_texts[0]).click()

In [12]:
page_soup = soup(driver.page_source, 'lxml')

In [13]:
containers = page_soup.findAll("h3", {"class": "background-rmc"})
for container in containers:
    if container.text.split()[0] == link_texts[0]:
        true_container = container
        
true_container

<h3 class="background-rmc" id="ene100" tabindex="-1">ENE100 Introduction to Literary Studies and University Writing Skills</h3>

In [14]:
course_code = true_container.text.split()[0]
course_code

'ENE100'

In [15]:
course_name = " ".join(true_container.text.split()[1:])
course_name

'Introduction to Literary Studies and University Writing Skills'

In [16]:
dir(true_container)

['__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_find_all',
 '_find_one',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descendant',
 '_should_pretty_print',
 'append',
 'attrs',
 'can_be_empty_element',
 'cdata_list_attributes',
 'childGenerator',
 'children',
 'clear',
 'contents',
 'decode',
 'decode_contents',
 'decompose',
 'decomposed',
 'descendants',
 'encode',
 'encode_contents',
 'extend',
 'extract',
 'fetchNextSiblings',
 'fetchParents',
 'fetchPrevious',
 'fetchPreviousSiblings',
 '

In [17]:
course_desc = true_container.findNext().text
course_desc

'This course provides an introduction to literary studies through a range of critical approaches, national perspectives, historical contexts, literary genres, and critical terms. Students will compose persuasive arguments that demonstrate close reading skills, logical reasoning, and a competence in writing at the university level.'

# 3. Test run successful. Implement automation script to scrape all courses

In [18]:
driver.get(url)

In [19]:
#take out language course for easier scraping
link_texts = [text for text in link_texts if text != "LCF"]

In [20]:
course_codes = []
course_names = []
course_descs = []
counter = 0

for link_text in link_texts:
    
    link = driver.find_element_by_link_text(link_text)
    time.sleep(2)
    link.click()
    time.sleep(3)
    
    page_soup = soup(driver.page_source, 'lxml')
    containers = page_soup.findAll("h3", {"class": "background-rmc"})
    for container in containers:
        if container.text.split()[0] == link_text:
            true_container = container
    
    course_codes.append(true_container.text.split()[0])
    course_names.append(" ".join(true_container.text.split()[1:]))
    course_descs.append(true_container.findNext().text)
    
    print("Scraped ", course_codes[-1])
    counter += 1
    
    driver.back()
    time.sleep(3)
    
print("Finished scraping {} courses".format(counter))

Scraped  ENE100
Scraped  PSE103
Scraped  CSE101
Scraped  CCE101
Scraped  MAE101
Scraped  PHE104
Scraped  ATE101
Scraped  MAE129
Scraped  GEE167
Scraped  HIE207
Scraped  MAE226
Scraped  PHE205
Scraped  MEE233
Scraped  GEE241
Scraped  GEE293
Scraped  HIE203
Scraped  POE205
Scraped  MAE209
Scraped  MAE227
Scraped  MEE231
Scraped  MEE245
Scraped  PSE301
Scraped  MAE328
Scraped  MEE311
Scraped  MEE331
Scraped  MEE333
Scraped  MEE351
Scraped  ATE301
Scraped  HIE271
Scraped  MEE301
Scraped  MEE303
Scraped  MEE313
Scraped  MEE346
Scraped  MEE353
Scraped  GEE393
Scraped  HIE289
Scraped  MEE471
Scraped  MEE407
Scraped  MEE421
Scraped  MEE431
Scraped  MEE443
Scraped  MEE482
Scraped  PSE401
Scraped  MEE401
Scraped  MEE404
Scraped  MEE417
Scraped  MEE423
Scraped  MEE425
Scraped  MEE435
Scraped  MEE437
Scraped  MEE438
Scraped  MEE446
Scraped  MEE451
Scraped  MEE469
Scraped  AEE435
Scraped  AEE461
Scraped  AEE463
Scraped  AEE467
Scraped  AEE491
Finished scraping 59 courses


In [21]:
len(link_texts)

59

# 4. Write to CSV

In [22]:
import pandas as pd

df = pd.DataFrame({
    
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
    
})

df

,Course Number,Course Name,Course Description
0,ENE100,Introduction to Literary Studies and Universit...,This course provides an introduction to litera...
1,PSE103,Introduction to Human Psychology,This course is designed to provide the student...
2,CSE101,Introduction to Algorithms and Computing,The subject of this course is the design and i...
3,CCE101,Introductory Chemistry,The course is designed to present the fundamen...
4,MAE101,Introductory Calculus,Fall term: Introduction to real numbers. Real ...
5,PHE104,General Physics,Introduction to the principles of physics invo...
6,ATE101,"Foundations of Fitness, Health and Sports",ATE101 is designed to provide students the too...
7,MAE129,Introduction to Algebra,"Introduction to sets. Vectors in 2, 3 and high..."
8,GEE167,Engineering Graphics I,The course introduces the students to the use ...
9,HIE207,History of Canada,"This course introduces students to the social,..."


In [23]:
df.to_csv('RMC_MechEng_Core_and_Electives_(AllYears)_Courses.csv', index = False)

In [24]:
driver.quit()